BM25

In [1]:
!pip install -U beir
!pip install 'elasticsearch<7.14.0'

     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 78 kB 3.6 MB/s 
     |████████████████████████████████| 8.4 MB 18.8 MB/s 
     |████████████████████████████████| 378 kB 65.5 MB/s 
     |████████████████████████████████| 4.4 MB 42.3 MB/s 
     |████████████████████████████████| 3.1 MB 29.8 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
     |████████████████████████████████| 1.2 MB 30.2 MB/s 
     |████████████████████████████████| 59 kB 7.2 MB/s 
     |████████████████████████████████| 895 kB 34.1 MB/s 
     |████████████████████████████████| 596 kB 69.2 MB/s 
  Created wheel for beir: filename=beir-0.2.3-py3-none-any.whl size=45805 sha256=eada6377ce3e2194adb349598e120eb0c6cc8db6d785fd499cddb50853486a88
  Stored in directory: /root/.cache/pip/wheels/af/29/4c/261c842753d34921b8b35c1b8679720114f8536654767430dd
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp37-cp37m-linux_x86_64.whl size=264594 sha256=869220763f

In [2]:
import pathlib, os
import time
import pandas as pd
from beir import util
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
if not os.path.exists('elasticsearch-oss-7.9.2-linux-x86_64.tar.gz'):
  !wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
  !wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
  !tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
  !sudo chown -R daemon:daemon elasticsearch-7.9.2/
  !shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512 

--2021-11-04 15:54:47--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229941304 (219M) [application/x-gzip]
Saving to: ‘elasticsearch-oss-7.9.2-linux-x86_64.tar.gz’

elasticsearch-oss-7 100%[===================>] 219.29M  7.60MB/s    in 35s     

2021-11-04 15:55:23 (6.28 MB/s) - ‘elasticsearch-oss-7.9.2-linux-x86_64.tar.gz’ saved [229941304/229941304]

--2021-11-04 15:55:23--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 20

In [4]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

Starting job # 0 in a separate thread.


In [5]:
time.sleep(20)

In [6]:
!ps -ef | grep elasticsearch

root         378     376  0 15:55 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       379     378 99 15:55 ?        00:00:20 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-4382450096319273423 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecou

In [7]:
!curl -sX GET "localhost:9200/"

{
  "name" : "68b674f374db",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "zws8zFUuST6Vo7hxOTdckw",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [8]:
def retrieve(dataset):
  hostname = 'localhost' 
  url = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip'
  out_dir = os.path.join(os.getcwd(), 'datasets')
  dataset_r = dataset
  index_r = dataset_r
  data_path_r = util.download_and_unzip(url.format(dataset_r), out_dir)
  corpus_r, queries_r, qrels_r = GenericDataLoader(data_path_r).load(split='test') # 'test', 'train', 'dev'
  model_r = BM25(index_name=index_r, hostname=hostname, initialize=True) # initialize=True : reindex
  retriever_r = EvaluateRetrieval(model_r)
  results_r = retriever_r.retrieve(corpus_r, queries_r)
  return retriever_r.evaluate(qrels_r, results_r, retriever_r.k_values)

In [9]:
def evaluate(dataset_name, model_name):
  ndcg, _map, recall, precision = retrieve(dataset_name)
  c_map = 'MAP@10'
  c_map_h = 'MAP@100'
  c_map_t = 'MAP@1000'
  c_ndcg = 'NDCG@10'
  c_ndcg_h = 'NDCG@100'
  c_ndcg_t = 'NDCG@1000'
  c_pre = 'P@10'
  c_pre_h = 'P@100'
  c_pre_t = 'P@1000'
  c_rec = 'Recall@10'
  c_rec_h = 'Recall@100'
  c_rec_t = 'Recall@1000'
  eval_dict = {
      c_map: [_map[c_map]], 
      c_map_h: [_map[c_map_h]],
      c_map_t: [_map[c_map_t]],
      c_ndcg: [ndcg[c_ndcg]], 
      c_ndcg_h: [ndcg[c_ndcg_h]],
      c_ndcg_t: [ndcg[c_ndcg_t]],
      c_pre: [precision[c_pre]], 
      c_pre_h: [precision[c_pre_h]],
      c_pre_t: [precision[c_pre_t]],
      c_rec: [recall[c_rec]],
      c_rec_h: [recall[c_rec_h]],
      c_rec_t: [recall[c_rec_t]]}
  eval_df = pd.DataFrame(data=eval_dict)
  eval_df.index = [model_name]
  return eval_df

**TREC-COVID** - [Homepage](https://ir.nist.gov/covidSubmit/index.html) - [Paper](https://arxiv.org/abs/2005.04474)

In [10]:
evaluate('trec-covid', 'TREC-COVID')

/content/datasets/trec-covid.zip:   0%|          | 0.00/70.5M [00:00<?, ?iB/s]

  0%|          | 0/171332 [00:00<?, ?it/s]

que: 100%|██████████| 1/1 [00:11<00:00, 11.37s/it]


,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
TREC-COVID,0.01698,0.08517,0.2084,0.68803,0.48454,0.433,0.734,0.5084,0.20194,0.01907,0.11731,0.41281


**SCIFACT** - [Homepage](https://allenai.org/data/scifact) - [Paper](https://arxiv.org/abs/2004.14974)

In [11]:
evaluate('scifact', 'SCIFACT')

/content/datasets/scifact.zip:   0%|          | 0.00/2.69M [00:00<?, ?iB/s]

  0%|          | 0/5183 [00:00<?, ?it/s]

que: 100%|██████████| 3/3 [00:17<00:00,  5.71s/it]


,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
SCIFACT,0.6386,0.64409,0.64438,0.68442,0.70769,0.71503,0.08967,0.01033,0.0011,0.80978,0.91256,0.97


**SCIDOCS** - [Homepage](https://allenai.org/data/scidocs) - [Paper](https://arxiv.org/abs/2004.07180)

In [12]:
evaluate('scidocs', 'SCIDOCS')

/content/datasets/scidocs.zip:   0%|          | 0.00/136M [00:00<?, ?iB/s]

  0%|          | 0/25657 [00:00<?, ?it/s]

que: 100%|██████████| 8/8 [00:52<00:00,  6.51s/it]


,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
SCIDOCS,0.0964,0.11199,0.11423,0.16471,0.23054,0.27383,0.0857,0.01811,0.00285,0.17372,0.36777,0.57852


**NFCORPUS** - [Homepage](https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/) - [Paper](https://www.cl.uni-heidelberg.de/~riezler/publications/papers/ECIR2016.pdf)

In [13]:
evaluate('nfcorpus', 'NFCORPUS')

/content/datasets/nfcorpus.zip:   0%|          | 0.00/2.34M [00:00<?, ?iB/s]

  0%|          | 0/3633 [00:00<?, ?it/s]

que: 100%|██████████| 3/3 [00:06<00:00,  2.13s/it]


,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
NFCORPUS,0.1306,0.15429,0.15949,0.34339,0.28744,0.3132,0.2461,0.06646,0.01104,0.16541,0.2553,0.3703


end of fun.